# loadShooterDesc Function

In [25]:
import os

def loadShooterDesc(directory_path):
   shooter_desc = {}
   
   for filename in os.listdir(directory_path):
      if filename.endswith(".txt"):  # assuming shooter_desc are in .txt files
         file_path = os.path.join(directory_path, filename)

         with open(file_path) as f:
            raw_shooter_desc = f.read()

         filename_without_ext = os.path.splitext(filename)[0]  # remove .txt extension
         shooter_desc[filename_without_ext] = [text.strip() for text in raw_shooter_desc.split('=====')]

   return shooter_desc


In [26]:
shooter_desc = loadShooterDesc('.')
shooter_desc

{'shooter_desc (4)': ["These are descriptions of a shooter or shooters.\nBlue Cap\nwearing sunglasses\nIt's a guy.\nWearing black jacket\nhas a gun"]}

In [27]:
docs = [{'text': filename + ' | ' + section, 'path': filename} for filename, sections in shooter_desc.items() for section in sections]

# Sample the resulting data
docs[:2]

[{'text': "shooter_desc (4) | These are descriptions of a shooter or shooters.\nBlue Cap\nwearing sunglasses\nIt's a guy.\nWearing black jacket\nhas a gun",
  'path': 'shooter_desc (4)'}]

In [28]:
un = "user1"
pw = "yoominchoi1234A"
cs = "localhost/FREEPDB1"

In [29]:
import oracledb
connection = oracledb.connect(user=un, password=pw, dsn=cs)

In [30]:
table_name = 'shooter_desc'

with connection.cursor() as cursor:
    # Create the table
    create_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id NUMBER PRIMARY KEY,
            payload CLOB CHECK (payload IS JSON),
            vector VECTOR
        )"""
    try:
        cursor.execute(create_table_sql)
    except oracledb.DatabaseError as e:
        raise

    connection.autocommit = True    

In [31]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer('all-MiniLM-L12-v2')

In [32]:
import array

# Define a list to store the data
data = [
    {"id": idx, "vector_source": row['text'], "payload": row}
    for idx, row in enumerate(docs)
]

# Collect all texts for batch encoding
texts = [f"{row['vector_source']}" for row in data]

# Encode all texts in a batch
embeddings = encoder.encode(texts, batch_size=32, show_progress_bar=True)

# Assign the embeddings back to your data structure
for row, embedding in zip(data, embeddings):
    row['vector'] = array.array("f", embedding)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
import json

with connection.cursor() as cursor:
    # Truncate the table
    cursor.execute(f"truncate table {table_name}")
    prepared_data = [(row['id'], json.dumps(row['payload']), row['vector']) for row in data]

    # Insert the data
    cursor.executemany(
        f"""INSERT INTO {table_name} (id, payload, vector)
            VALUES (:1, :2, :3)""", prepared_data)
    
    connection.commit()

In [34]:
with connection.cursor() as cursor:
    # Define the query to select all rows from a table
    query = f"SELECT * FROM {table_name}"

    # Execute the query
    cursor.execute(query)

    # Fetch all rows
    rows = cursor.fetchall()

    # Print the rows
    for row in rows[:5]:
        print(row)

(0, {'text': "shooter_desc (4) | These are descriptions of a shooter or shooters.\nBlue Cap\nwearing sunglasses\nIt's a guy.\nWearing black jacket\nhas a gun", 'path': 'shooter_desc (4)'}, array('f', [-0.017141368240118027, 0.04507311433553696, -0.05348360911011696, -0.04635532200336456, 0.11879650503396988, 0.029685821384191513, 0.1721850484609604, -0.008428659290075302, -0.09266557544469833, 0.11536173522472382, 0.03890695050358772, -0.0596795491874218, 0.02103598788380623, -0.051024384796619415, 0.03891414403915405, -0.011247022077441216, -0.026512511074543, -0.01963193342089653, 0.04332781955599785, -0.0716574490070343, -0.07306758314371109, -0.06262847781181335, 0.0029959452804178, -0.05905653536319733, -0.05675629898905754, -0.06291121989488602, 0.049255236983299255, -0.011615409515798092, -0.06431914120912552, 0.06185390055179596, 0.09699669480323792, -0.02571626380085945, -0.012552118860185146, -0.042645640671253204, -0.028517138212919235, -0.031332287937402725, 0.0103616816923

# Vector retrieval and Large Language Model generation
## 1) Vectorize the "question"

In [35]:
topK = 3

sql = f"""select payload, vector_distance(vector, :vector, COSINE) as score
from {table_name}
order by score
fetch approx first {topK} rows only"""

In [36]:
question = 'Make a sentence of a description of the shooter.'

In [37]:
with connection.cursor() as cursor:
    embedding = list(encoder.encode(question))
    vector = array.array("f", embedding)
    
    results  = []
    
    for (info, score, ) in cursor.execute(sql, vector=vector):
        text_content = info.read()
        results.append((score, json.loads(text_content)))

In [38]:
import pprint
pprint.pp(results)

[(0.4035865621903405,
  {'text': 'shooter_desc (4) | These are descriptions of a shooter or '
           'shooters.\n'
           'Blue Cap\n'
           'wearing sunglasses\n'
           "It's a guy.\n"
           'Wearing black jacket\n'
           'has a gun',
   'path': 'shooter_desc (4)'})]


## 2) Create the LLM prompt

In [39]:
from transformers import LlamaTokenizerFast
import sys

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")
tokenizer.model_max_length = sys.maxsize

def truncate_string(string, max_tokens):
    # Tokenize the text and count the tokens
    tokens = tokenizer.encode(string, add_special_tokens=True) 
    # Truncate the tokens to a maximum length
    truncated_tokens = tokens[:max_tokens]
    # transform the tokens back to text
    truncated_text = tokenizer.decode(truncated_tokens)
    return truncated_text

In [40]:
# transform docs into a string array using the "paylod" key
docs_as_one_string = "\n=========\n".join([doc["text"] for doc in docs])
docs_truncated = truncate_string(docs_as_one_string, 1000)

In [41]:
prompt = f"""\
    <s>[INST] <<SYS>>
    You are a helpful assistant named Oracle chatbot. 
    USE ONLY the sources below and ABSOLUTELY IGNORE any previous knowledge.
    Use Markdown if appropriate.
    Assume the customer is highly technical.
    <</SYS>> [/INST]

    [INST]
    Respond to PRECISELY to this question: "{question}.",  USING ONLY the following information and IGNORING ANY PREVIOUS KNOWLEDGE.
    Include code snippets and commands where necessary.
    NEVER mention the sources, always respond as if you have that knowledge yourself. Do NOT provide warnings or disclaimers.
    =====
    Sources: {docs_truncated}
    =====
    Answer (Three paragraphs, maximum 200 words each, 90% spartan):
    [/INST]
    """

## 3) Call the Generative AI Service LLM

In [42]:
import oci

compartment_id = 'ocid1.tenancy.oc1..aaaaaaaaj4ccqe763dizkrcdbs5x7ufvmmojd24mb6utvkymyo4xwxyv3gfa'
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('config', CONFIG_PROFILE)

# Service endpoint
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

# GenAI client
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))

In [43]:
generate_text_request = oci.generative_ai_inference.models.LlamaLlmInferenceRequest()

generate_text_request.prompt = prompt
generate_text_request.is_stream = False #SDK doesn't support streaming responses, feature is under development
generate_text_request.maxx_tokens = 1000
generate_text_request.temperature = 0.3
generate_text_request.top_p = 0.7
generate_text_request.frequency_penalty = 0.0

generate_text_detail = oci.generative_ai_inference.models.GenerateTextDetails()
generate_text_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id="meta.llama-2-70b-chat")
generate_text_detail.compartment_id = compartment_id
generate_text_detail.inference_request = generate_text_request

generate_text_response = generative_ai_inference_client.generate_text(generate_text_detail)
response = generate_text_response.data.inference_response.choices[0].text

print(response.strip())

Based on the information provided, here is a possible description of the shooter:
